In [15]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

설치된 openai 의 버전을 체크합니다.


## Gemini

Google 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://gemini.google.com/app

- install
```sh
# pip install -q -U google-generativeai
pip install -q --upgrade google-generativeai langchain-google-genai
```


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='서울' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-ecfd2f16-f88e-4b16-a18e-1c699b5a5209-0'


## 프롬프트 템플릿의 활용


`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.


In [17]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체


`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다
- 사용법
  - `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
  - `llm`: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.


In [18]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [19]:
llm_chain.invoke({"country": "대한민국"})

{'country': '대한민국', 'text': '서울'}

In [20]:
llm_chain.invoke({"country": "캐나다"})

{'country': '캐나다', 'text': '오타와'}

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [21]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

response = llm_chain.apply(input_list)

In [22]:
response[0]["text"]

'캔버라'

`text` 키 값으로 결과 뭉치가 반환되었음을 확인할 수 있습니다.

이를 반복문으로 출력한다면 다음과 같습니다.


In [23]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

캔버라
베이징
암스테르담


### `generate()`


`generate()` 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.


In [24]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='캔버라', generation_info={'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, message=AIMessage(content='캔버라', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': Fals

In [25]:
# 토큰 사용량 출력
generated_result.llm_output

{}

### 2개 이상의 변수를 템플릿 안에 정의


2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(`input_variables`) 를 활용하여 템플릿 구성을 해보겠습니다.


In [26]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [27]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [28]:
# 체인 실행: run()
print(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

{'area1': '서울', 'area2': '파리', 'text': '8시간'}


In [29]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

6시간
20시간
2시간


## stream: 실시간 출력

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 `streaming=True` 로 설정하고 스트리밍으로 답변을 받기 위한 `StreamingStdOutCallbackHandler()` 을 콜백으로 지정합니다.


In [30]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
# llm = ChatOpenAI(
#     temperature=0,  # 창의성 (0.0 ~ 2.0)
#     max_tokens=2048,  # 최대 토큰수
#     model_name="gpt-3.5-turbo",  # 모델명
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()],
# )

In [14]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)